In [0]:
import logging

In [0]:
# Autenticator
def autenticator(storage_account_key, logger):
    try:
        # Configure a credencial no spark
        spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", storage_account_key)

        return logger.info(f"Authentication carried out successfully")
    
    except ValueError as e:
        return logger.error(f"Authentication failed: {e}")

#List and delete
def list_and_delete_files(container, storage_account_name, path, logger):
    try:
        # Listar arquivos
        files_df = spark.read.format("binaryFile").load(path)
        files_list = files_df.select("path").rdd.flatMap(lambda x: x).collect()
        
        if not files_list:
            logger.info("No files found to delete.")
            return
        
        logger.info(f"Files found: {files_list}")
        
        # Apagar arquivos
        for file_path in files_list:
            if dbutils.fs.ls(file_path):
                dbutils.fs.rm(file_path, True)
                logger.info(f"Deleted file: {file_path}")
        
    except Exception as e:
        logger.error(f"Error while listing or deleting files: {e}")


In [0]:
# Configurar logger
logger = logging.getLogger("ExpurgoStageBlobStorage")
logger.setLevel(logging.INFO)
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

# Defina o nome do contêiner e o nome da conta de armazenamento
storage_account_name = "stagedatamaster"
sas_token = dbutils.secrets.get(scope="storage_datamaster", key="data_master")
storage_account_key = dbutils.secrets.get(scope="stage_datamaster", key="stage_datamaster_key")

# Autenticar
autenticator(storage_account_key, logger)

container = dbutils.widgets.get("param1")
table = dbutils.widgets.get("param2")
dat_carga_origin = dbutils.widgets.get("param3")

logger.info(f"Container: {container}")
logger.info(f"Table: {table}")
logger.info(f"Dat Carga Origin: {dat_carga_origin}")

path = f"abfss://{container}@{storage_account_name}.dfs.core.windows.net/table_ingestion_files/{table}/{dat_carga_origin}"

# Listar e apagar arquivos
list_and_delete_files(container, storage_account_name, path, logger)